In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import unidecode

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
URL =  'https://www.onpe.gob.pe/elecciones/historico-elecciones/'
driver.get( URL )
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Usuario\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
# confirmando que estoy en la página indicada
print(driver.title)
print(driver.current_url)

Histórico de Elecciones
https://www.onpe.gob.pe/elecciones/historico-elecciones/


In [4]:
VOTES = pd.DataFrame(columns=['political_parties', 'total', 'share_valid', 'share_emited', 'Continent', 'Country', 'Estate', 'Ubigeo', 'Link'])

In [5]:
driver.find_element_by_xpath( "/html/body/div/div/div/section[2]/div/div[2]/div/div/div[2]/div/div[2]/a").click()
# See all tabs
driver.window_handles
# Go to the second tab
driver.switch_to.window(driver.window_handles[1])
# Click on Results
driver.find_element_by_xpath( "/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a" ).click()

# Give time to the page to refresh
time.sleep( 4 )

# Go to Extranjeros
scope = Select(driver.find_element_by_id( "select_ambito" ))
dict_scope_options = { option.text : option for option in scope.options }
dict_scope_options['EXTRANJERO'].click()

continent = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select'))
for continent_idx in range(len(continent.options)):
    continent = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select')).options[continent_idx]
    continent_name = continent.text

    if continent_name != "--TODOS--":
        continent.click()
        country = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select'))
        # Rodrigo
        for country_idx in range(len(country.options)):
            country = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select')).options[country_idx]
            country_name = country.text

            if country_name != "--TODOS--":
                country.click()
                # Claudia
                estate = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select'))

                for estate_idx in range(len(estate.options)):
                    estate = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select')).options[estate_idx]
                    estate_name = estate.text

                    if estate_name != "--TODOS--":
                        ubigeo = driver.current_url.split("/")[-1]
                        link = driver.current_url
                        estate.click()
                        time.sleep(2)
                        #Rodrigo2
                        table_path = driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]')
                        table_html = table_path.get_attribute('innerHTML')
                        table = pd.read_html(table_html)
                        table_ = table[0].iloc[ 1:, 2: ]
                        table_.columns = ['political_parties', 'total', 'share_valid', 'share_emited']
                        table_.fillna('0.000%')
                        table_['Continent'] = continent_name
                        table_['Country'] = country_name
                        table_['Estate'] = estate_name
                        table_['Ubigeo'] = ubigeo
                        table_['Link'] = link
                        #Daniel
                        VOTES = pd.concat([VOTES, table_], axis=0)
                        VOTES.reset_index(inplace=True, drop=True)
                        

In [8]:
VOTES.to_excel(  r'../_data_results/group_3_ass_4.xlsx' , index = False )